In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
from tqdm import tqdm
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout
from tensorflow.keras.callbacks import EarlyStopping
import tensorflow as tf
import random

In [ ]:
DATA_DIR = "spectograms_nine_emotions"

In [ ]:
CATEGORIES = ["Neutral","Anxiety", "Fear",  "Disgust", "Happy", "Irritation", "Relief", "Sad", "Surprise"]

In [ ]:
for category in CATEGORIES:  # do positive and negative
    path = os.path.join(DATA_DIR,category)  # create path to positive and negative
    for img in os.listdir(path):  # iterate over each spectogram per positive and negative
        image = cv2.imread(os.path.join(path,img))  # convert to array
        image_array = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
#         plt.imshow(image_array, cmap='gray')
#         plt.show() 
#         print(image_array.shape)
        break
    break

In [ ]:
print(image_array.shape)

In [ ]:
training_data = []


for category in CATEGORIES:  # do positive and negative
    path = os.path.join(DATA_DIR,category)  # create path to positive and negative
    for img in os.listdir(path):  # iterate over each spectogram per positive and negative
        image = cv2.imread(os.path.join(path,img))
        image_array = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

def create_training_data():
    for category in CATEGORIES:  # do positive and negative
        path = os.path.join(DATA_DIR,category)  # create path to positive and negative
        class_num = CATEGORIES.index(category)  # get the classification  (0 or a 1). 0=Positive 1=Negative

        for img in tqdm(os.listdir(path)):  # iterate over each image per positive and negative
            try:
                image = cv2.imread(os.path.join(path,img))
                image_array = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
                training_data.append([image_array, class_num])  # add this to our training_data
            except Exception as e:  # in the interest in keeping the output clean...
                pass

create_training_data()

print(len(training_data))

In [ ]:
random.shuffle(training_data)

for sample in training_data[:10]:
    print(sample[1])

In [ ]:
X = []
y = []

for features,label in training_data:
    X.append(features)
    y.append(label)

In [ ]:
X = np.array(X)
y = np.array(y)

In [ ]:
# np.save('X_valence.npy', X)
# np.save('Y_valence.npy', y)

In [ ]:
X = X/255.0

In [ ]:
X.shape

In [ ]:
y.shape

In [ ]:
# creating train data and testing data
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

In [ ]:
model = Sequential()

model.add(Conv2D(16, (3,3), 1, activation='relu', input_shape=X.shape[1:]))
model.add(MaxPooling2D())

model.add(Conv2D(32, (3,3), 1, activation='relu'))
model.add(MaxPooling2D())

model.add(Conv2D(16, (3,3), 1, activation='relu'))
model.add(MaxPooling2D())

model.add(Flatten())

model.add(Dense(128, activation='relu'))
model.add(Dense(9, activation='softmax'))

In [ ]:
# compile model
model.compile(optimizer='adam', loss=tf.losses.sparse_categorical_crossentropy, metrics=['accuracy'])

In [ ]:
# define early stopping callback
early_stop = EarlyStopping(monitor='val_loss', patience=5)

In [ ]:
logdir='logs2'
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir)

In [ ]:
# train model
hist = model.fit(x_train, y_train, epochs=20, verbose=1, validation_split=0.2, callbacks=[early_stop, tensorboard_callback])

In [ ]:
# evaluate model
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

In [ ]:
fig = plt.figure()
plt.plot(hist.history['loss'], color='teal', label='loss')
plt.plot(hist.history['val_loss'], color='orange', label='val_loss')
fig.suptitle('Loss', fontsize=20)
plt.legend(loc="upper left")
plt.show()

In [ ]:
from sklearn.metrics import confusion_matrix

# make predictions on test data
y_pred = model.predict(x_test)

# convert predicted probabilities to class labels
y_pred_labels = np.argmax(y_pred, axis=1)
print(y_pred_labels)
print(y_test)
# # convert true labels to class labels
# y_true_labels = np.argmax(y_test, axis=1)

# create confusion matrix
confusion_mtx = confusion_matrix(y_test, y_pred_labels)

# print confusion matrix
print(confusion_mtx)

In [ ]:
import seaborn as sns
# define class labels
class_names = ["Neutral","Anxiety", "Fear",  "Disgust", "Happy", "Irritation", "Relief", "Sad", "Surprise"]

# create heatmap
sns.heatmap(confusion_mtx, annot=True, fmt='d', xticklabels=class_names, yticklabels=class_names)

# add labels and title
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')

# show plot
plt.show()